In [39]:
import os
import time
import importlib
import tqdm

from alpha_vantage.timeseries import TimeSeries
from alpha_vantage.techindicators import TechIndicators

import common
importlib.reload(common)
from common import chech_if_should_buy

## Prepare API

In [40]:
key = 'MXSJTESP5AWNK9TI'

In [41]:
ti = TechIndicators(key, output_format='pandas')

## Get S&P 500 companies' symbols

In [42]:
with open('../sp_500_symbols.txt', 'r') as f:
    sp_500_symbols = f.read().split('\n')[:-1]

In [43]:
print(len(sp_500_symbols))
print(sp_500_symbols[:5])

505
['MMM', 'AOS', 'ABT', 'ABBV', 'ACN']


## Identify stocks to buy 

In [44]:
def identify_stocks_to_buy(ti, symbols):
    stocks_to_buy_ti = []
    stocks_to_buy = []
    for stock in tqdm.tqdm(symbols):
        stock_ti_df, should_buy = chech_if_should_buy(ti, stock)
        if should_buy:
            stocks_to_buy_ti.append(stock_ti_df)
            stocks_to_buy.append(stock)
            print(stock)
        time.sleep(60)  # API calls frequency restrictions
        
    return stocks_to_buy_ti, stocks_to_buy

In [ ]:
stocks_to_buy_ti, stocks_to_buy = identify_stocks_to_buy(ti, sp_500_symbols)

  1%|▏         | 7/505 [07:40<9:05:25, 65.71s/it]